## AMR in genomes from mysterious city
First lets upload the mysterious city AMR presence-absence table
We have 143 genomes (rows) with 505 AMR markers detected

We have many tables of CARD-manual. Since mysterious samples just register presence and absence we divide our tables in Count tables and Presence tables.

The list of such tables are in: https://github.com/FranciscoSNR/camda2023/tree/main/Taxonomy
This table were made by Anthon.

Todas las tablas poseen las siguientes columnas:
Genus . Género predicho por RGI (el anotador de genes de resistencia utilizado).
Gene . Nombre del gen de resistencia.
ARO . Identificador único del gen en la base de datos de CARD.
El resto de las columnas corresponden a los ensamblados analizados.
Las tablas están clasificadas en cuatro categorías:

SampleAssemblies . Genes de resistencia en ensamblados por muestra. El formato de las
columnas es AABBBCC , done AA corresponde al año, BBB es el nombre de la ciudad con tres letras, y
CC es el número de muestra.

Coassemblies . Genes de resistencia en coensamblados por ciudad. El formato de las columnas es
BBB (es decir, el nombre de la ciudad con tres letras).

CoassembliesExtracted . Genes de resistencia en los contigs extraídos de los coensamblados
por ciudad que Kraken2 clasificó como Enterobacter, Escherichia o Klebsiella. El formato de las
columnas es BBB_Dd , donde BBB es el nombre de tres letras de la ciudad y Dd es el nombre de la
especie ( En : Enterobacter, Es : Escherichia, Kl : Klebsiella)

ExtractedReadAssemblies . Genes de resistencia en los ensamblados de las lecturas por ciudad
que Kraken2 clasificó como Enterobacter, Escherichia o Klebsiella. El formato del nombre de las
columnas es idéntico al de CoassembliesExtracted .

Nota. Las tablas de Coassemblies , CoassembliesExtracted y ExtractedReadAssemblies
sólo incluyen ciudades de Estados Unidos.
Las tablas de SampleAssemblies incluyen a todas las ciudades.

Cada una de estas categorías posee dos tablas marcadas como Strict y dos como Full :

Las tablas Strict contienen genes de resistencia que RGI identificó como confiables.

Las tablas Full incluyen a todos los genes de resistencia, incluyendo aquellos cuyo nivel de
confianza es menor.

Las tablas también se encuentran clasificadas en Counts y en Presence :
Las tablas Counts guardan los conteos de cada gen por ensamble analizado.
Las tablas Presence almacenan las presencias (1s) y ausencias (0s) por ensamble analizado. Se
contruyeron a partir de las tablas de Counts : se marcan como presentes aquéllos genes que están
presentes por lo menos una vez.

In [1]:
import pandas as pd
## 505 combined CARD-manual Ids from mysterious samples   ### This table is incorrect, more ones are required by roow
url = 'https://raw.githubusercontent.com/ccm-bioinfo/cambda2023/main/06_amr_resistance/data/amr_mistery_table20230622_aroIds.tsv'
# Read the TSV file into a DataFrame
df_CM_mysterious = pd.read_csv(url, delimiter='\t')
# Rename the first column
df_CM_mysterious.columns.values[0] = 'ID'

# Print the DataFrame
print(df_CM_mysterious.head())

# Get the dimensions of the DataFrame
rows_df_CM_mysterious, columns_df_CM_mysterious = df_CM_mysterious.shape

# Print the dimensions
print("Number of rows:", rows_df_CM_mysterious)
print("Number of columns:", columns_df_CM_mysterious)
# Count non-zero values by row
non_zero_counts = (df_CM_mysterious != 0).sum(axis=1)

# Print the result
print(non_zero_counts)
#print (df)


   ID  ARO:3000167  ARO:3002847  mrkb  enta  fepa  ARO:3004122  cyca  csge  \
0  S1            0            0     0     0     0            0     1     0   
1  S2            0            0     0     0     0            0     1     0   
2  S3            0            0     0     0     0            0     1     0   
3  S4            0            0     0     1     0            0     1     0   
4  S5            0            0     0     0     0            0     1     0   

   iucc  ...  teml100  scintssj  wcst  sfax  kpsd  mdlb  ARO:3001084  tssj  \
0     0  ...        0         0     0     0     0     0            0     0   
1     0  ...        0         0     0     0     0     0            0     0   
2     0  ...        0         0     0     0     0     0            0     0   
3     0  ...        0         0     0     0     0     0            0     0   
4     0  ...        0         0     0     0     0     0            0     0   

   icmftssm  acrr  
0         0     0  
1         0     0  
2 

The same datafram with spices. Then add a column called city

In [2]:
## Adding metadata from such as species to mysterious samples
url = 'https://raw.githubusercontent.com/ccm-bioinfo/cambda2023/main/06_amr_resistance/codigos/data_preparation/data/amr_patterns.tsv'  # Replace with the actual GitHub URL of the CSV file

# Read the TSV file into a DataFrame
df_metadata_mysterious = pd.read_csv(url, delimiter='\t')
new_column = df_metadata_mysterious['ID']
df_metadata_mysterious.insert(2, 'City', new_column)
df_metadata_mysterious['City'] = 'mysterious'
df_metadata_mysterious=df_metadata_mysterious.iloc[:, :4]

# Print the DataFrame
#print(df_metadata_mysterious.head())
# Perform the join based on the 'Id' column
mysterious_df = pd.merge( df_metadata_mysterious,df_CM_mysterious, on='ID')


# Print the merged DataFrame
print(mysterious_df.head())

   ID                  Species        City AST-based group  ARO:3000167  \
0  S1  Enterobacter hormaechei  mysterious   3GC-resistant            0   
1  S2  Enterobacter hormaechei  mysterious          CP CRE            0   
2  S3  Enterobacter hormaechei  mysterious          CP CRE            0   
3  S4  Enterobacter hormaechei  mysterious          CP CRE            0   
4  S5  Enterobacter hormaechei  mysterious          CP CRE            0   

   ARO:3002847  mrkb  enta  fepa  ARO:3004122  ...  teml100  scintssj  wcst  \
0            0     0     0     0            0  ...        0         0     0   
1            0     0     0     0            0  ...        0         0     0   
2            0     0     0     0            0  ...        0         0     0   
3            0     0     1     0            0  ...        0         0     0   
4            0     0     0     0            0  ...        0         0     0   

   sfax  kpsd  mdlb  ARO:3001084  tssj  icmftssm  acrr  
0     0     0    

## AMR in metagenomes  
### Extracted by taxonomy - Binned by city - Assembled - AMR profiled

As we mantion before, there are many AMR-CARD tables that were gotten with diferent methods.
Let explore some of them.

First we focus on precense and absent tables because mysterius sample is presented in that way. 


%We found 492 CARD Ids ( columns) not all of them are in the mysterious city samples.  
%The 12 rows are the genus _Escherichia_, _Enterobacter_ and _Klebsiella_ in the 6 US cities were AMR CARD models were found.




We start with Coassemblies_Strict_Presence

In [3]:
# CARD table for Extracted by Taxonomy , binned by city
url = 'https://raw.githubusercontent.com/FranciscoSNR/camda2023/main/Taxonomy/Coassemblies/Coassemblies_Strict_Presence.tsv' 
# Read the TSV file into a DataFrame


df_ET_BC_AMR_CARD = pd.read_csv(url, delimiter='\t').transpose()

# Get the values from the third row
third_row = df_ET_BC_AMR_CARD.iloc[2]
#print(third_row)
df_ET_BC_AMR_CARD.columns=third_row

# Add a prefix to column names
prefix = 'ARO:'  # Replace with the desired prefix
df_ET_BC_AMR_CARD = df_ET_BC_AMR_CARD.add_prefix(prefix)

### Las part was no required this time.
# Remove a specific row by index
index_to_remove = "ARO"  # Replace with the index of the row you want to remove
df_ET_BC_AMR_CARD = df_ET_BC_AMR_CARD.drop(index=index_to_remove)

print(df_ET_BC_AMR_CARD.head())

ARO     ARO:3001821   ARO:3001842   ARO:3006267   ARO:3006268   ARO:3001831  \
Genus  Enterobacter  Enterobacter  Enterobacter  Enterobacter  Enterobacter   
Gene          ACT-1        ACT-21        ACT-73        ACT-74         ACT-8   
BAL               0             0             0             0             0   
DEN               0             0             0             0             0   
MIN               0             0             0             0             0   

ARO     ARO:3000518                ARO:3004042  \
Genus  Enterobacter               Enterobacter   
Gene            CRP  Enterobacter cloacae acrA   
BAL               0                          0   
DEN               0                          0   
MIN               0                          0   

ARO                                          ARO:3003369  \
Genus                                       Enterobacter   
Gene   Escherichia coli EF-Tu mutants conferring resi...   
BAL                                         

Now we will get the CARD Ids intersection with IDs in mysterious city samples, only 57 were found.  

In [4]:
# Create an empty column with the desired name
new_column = df_ET_BC_AMR_CARD.index

# Get the intersection of column names
common_columns = list(set(df_CM_mysterious.columns).intersection(set(df_ET_BC_AMR_CARD.columns)))

# Keep only the common columns in both DataFrames
#df1 = df_CM_mysterious[common_columns]
df_ET_BC_AMR_CARD = df_ET_BC_AMR_CARD[common_columns]

# Print the updated DataFrames with only the common columns
# Assign the empty column as the first column in the DataFrame
df_ET_BC_AMR_CARD.insert(0, 'db', new_column)
df_ET_BC_AMR_CARD=df_ET_BC_AMR_CARD.reset_index(drop=True)

print(df_ET_BC_AMR_CARD.head())

ARO     db   ARO:3000516 ARO:3000516  ARO:3000792  ARO:3003550  \
0    Genus  Enterobacter  Klebsiella  Escherichia  Escherichia   
1     Gene          emrR        emrR         mdtA         mdtP   
2      BAL             1           0            0            0   
3      DEN             0           1            0            0   
4      MIN             0           0            0            0   

ARO                 ARO:3004588  ARO:3000027   ARO:3000823  ARO:3000833  \
0                    Klebsiella  Escherichia  Enterobacter  Escherichia   
1    Klebsiella pneumoniae KpnG         emrA          ramA         evgS   
2                             0            0             0            0   
3                             1            0             0            0   
4                             0            0             0            0   

ARO   ARO:3000676  ...  ARO:3001216 ARO:3001216   ARO:3000828 ARO:3000828  \
0    Enterobacter  ...  Escherichia  Klebsiella  Enterobacter  Klebsiella  

Now we want to load the blast table

In [5]:
url2='https://raw.githubusercontent.com/ccm-bioinfo/cambda2023/main/06_amr_resistance/data/amr_counts_complement_b3_20230629.tsv'
# this link is the result given by blast our 325 ids database vs cambda metagenomic samples


# Read the TSV file into a DataFrame
df_ET_BC_AMR_BLASTn = pd.read_csv(url2, delimiter='\t')
df_ET_BC_AMR_BLASTn['db'] = df_ET_BC_AMR_BLASTn['db'].str.split('.').str[0]

# Create an empty column with the desired name
new_column = df_ET_BC_AMR_BLASTn['db']

# Print the DataFrame
#print(df_ET_BC_AMR_BLASTn)

# Get the intersection of column names
common_columns = list(set(df_CM_mysterious.columns).intersection(df_ET_BC_AMR_BLASTn))

# Keep only the common columns in both DataFrames
#df1 = df_CM_mysterious[common_columns]
df_ET_BC_AMR_BLASTn= df_ET_BC_AMR_BLASTn[common_columns]
# Transform DataFrame to boolean
df_boolean = df_ET_BC_AMR_BLASTn.astype(bool)
# Convert boolean values back to numeric
df_ET_BC_AMR_BLASTn = df_boolean.astype(int)


# Assign the empty column as the first column in the DataFrame
df_ET_BC_AMR_BLASTn.insert(0, 'db', new_column)
df_ET_BC_AMR_BLASTn=df_ET_BC_AMR_BLASTn.reset_index(drop=True)

# Print the updated DataFrames with only the common columns
#print(df1)
print(df_ET_BC_AMR_BLASTn.head())

       db  tssa  kpsm  gspm  ykgkecpr  wcag  hcp2tssd2  kpse  \
0  BAL_En     0     0     0         0     0          0     0   
1  BAL_Es     0     0     0         0     0          0     0   
2  BAL_Kl     0     0     0         0     0          0     0   
3  DEN_En     0     0     0         0     0          0     0   
4  DEN_Es     0     0     0         0     0          0     0   

   aac630_aac6ibfusionprotein  hlyd  ...  chuy  flii  papk  papj  mrkh  flia  \
0                           0     0  ...     0     1     0     0     0     0   
1                           0     0  ...     0     0     0     0     0     0   
2                           0     0  ...     0     0     0     0     0     0   
3                           0     0  ...     0     0     0     0     0     0   
4                           0     0  ...     0     0     0     0     0     0   

   ybts  clbo  ibea  papc  
0     0     0     0     0  
1     0     0     0     0  
2     0     0     0     0  
3     0     0     0   

We want to joint the AMR_BLASTn table and AMR_CARD table horizontaly. However the structure of our tables are diferent.

We will transform the AMR_CARD table

Our problem is that the have diferenmts formats. We want secondone looks like firstone.

In particular :

1) the first column has the name "db" and it has 18 rows of the form city_spice

In [6]:
data=[]
new_row_En=pd.DataFrame(data)
for c in range(2,8):
    city=df_ET_BC_AMR_CARD["db"]
    new_row_En["db"]=city[c]+"_"+"En"
    data.append(city[c]+"_"+"En")
    new_row_En["db"]=city[c]+"_"+"Es"
    data.append(city[c]+"_"+"Es")
    new_row_En["db"]=city[c]+"_"+"Kl"
    data.append(city[c]+"_"+"Kl")
df_ET_BC_AMR_CARD_mod=pd.DataFrame(data)
df_ET_BC_AMR_CARD_mod=df_ET_BC_AMR_CARD_mod.rename(columns={0: 'db'})
#print(df_ET_BC_AMR_CARD_mod)

Got it

Now We need to spllit range of 18 in 3 sets

In [7]:
from collections import defaultdict

# Initialize a defaultdict to group numbers based on their remainder when divided by 3
grouped_nums = defaultdict(list)

# Divide range(18) into 3 sets based on remainder when divided by 3
for num in range(18):
    remainder = num % 3
    grouped_nums[remainder].append(num)
List_num=list(grouped_nums.items())
List_0=List_num[0][1]
List_1=List_num[1][1]
List_2=List_num[2][1]
print(List_0,List_1,List_2)

[0, 3, 6, 9, 12, 15] [1, 4, 7, 10, 13, 16] [2, 5, 8, 11, 14, 17]


2) We want a column for each gene in the 3th roe in the table modified

In [8]:
names_of_col=[]
new_col_zeros=[]
for i in range(18):
    new_col_zeros.append(0)
    
for i in range(1,len(df_ET_BC_AMR_CARD.columns)):
    new_col=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    #new_col=new_col_ceros
    #print("new_col_ceros  entering for",new_col_ceros)
    #print("new_col  entering for",new_col)

    if df_ET_BC_AMR_CARD.iloc[0,i]=="Enterobacter":
        modulo=0
    elif df_ET_BC_AMR_CARD.iloc[0,i]=="Escherichia":
        modulo=1
    elif df_ET_BC_AMR_CARD.iloc[0,i]=="Klebsiella":
        modulo=2
        
    for j in range(6):
        if modulo==0:
            if  df_ET_BC_AMR_CARD.iloc[j+2,i]==1:
                new_col[List_0[j]]=1
            else:
                new_col[List_0[j]]=0
        elif modulo==1:
            if  df_ET_BC_AMR_CARD.iloc[j+2,i]==1:
                new_col[List_1[j]]=1
            else:
                new_col[List_0[j]]=0
        elif modulo==2:
            if  df_ET_BC_AMR_CARD.iloc[j+2,i]==1:
                new_col[List_2[j]]=1
            else:
                new_col[List_0[j]]=0
    #print("new_col", new_col)                
    df_ET_BC_AMR_CARD_mod[df_ET_BC_AMR_CARD.columns[i]+"_"+str(i)]=new_col
        
        #restart the column with zeros
    #print("new_col_ceros",new_col_ceros)

    new_col=new_col_zeros
    #print("ne_col after equal to new_col_ceros",new_col)
    #new_col.append(df_ET_BC_AMR_CARD.iloc[2,i]+"_"+i)
    #if df_ET_BC_AMR_CARD.iloc[2,i] not in names_of_col:
    #    names_of_col.append(df_ET_BC_AMR_CARD.iloc[2,i])
print(df_ET_BC_AMR_CARD_mod.head())


       db  ARO:3000516_1  ARO:3000516_2  ARO:3000792_3  ARO:3003550_4  \
0  BAL_En              1              0              0              0   
1  BAL_Es              0              0              0              0   
2  BAL_Kl              0              0              0              0   
3  DEN_En              0              0              0              0   
4  DEN_Es              0              0              0              0   

   ARO:3004588_5  ARO:3000027_6  ARO:3000823_7  ARO:3000833_8  ARO:3000676_9  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

   ...  ARO:3001216_49  ARO:3001216_50  ARO:3000828_51  ARO:3000828_52  \
0  ...

we will to "colaps" or sum the columns by gene. if 2 columns start in the same way (x), sum them and get a new col called x delait the columns of the form "x_number" and at "x" to the end.

get the name of the genes

In [9]:
List_of_names=[]
for name in df_ET_BC_AMR_CARD_mod.columns[1:]:
    real_name=name.split("_")[0]
    if real_name in List_of_names:
        True
    else:
        List_of_names.append(real_name)
#print(List_of_names)


#redo the first column as before
data=[]
new_row_En=pd.DataFrame(data)
#CARD=pd.DataFrame(data)
for c in range(2,8):
    city=df_ET_BC_AMR_CARD["db"]
    new_row_En["db"]=city[c]+"_"+"En"
    data.append(city[c]+"_"+"En")
    new_row_En["db"]=city[c]+"_"+"Es"
    data.append(city[c]+"_"+"Es")
    new_row_En["db"]=city[c]+"_"+"Kl"
    data.append(city[c]+"_"+"Kl")
df_ET_BC_AMR_CARD_mod_2=pd.DataFrame(data)
df_ET_BC_AMR_CARD_mod_2=df_ET_BC_AMR_CARD_mod.rename(columns={0: 'db'})


#collect the columns that start the same, add them, add them to the end
for real_name in List_of_names:
    for name in df_ET_BC_AMR_CARD_mod.columns[1:]:
        if real_name==name.split("_")[0]:
            real_name_columns = [col for col in df_ET_BC_AMR_CARD_mod.columns[1:] if col.startswith(real_name)]
            sum_of_real_name_columns=df_ET_BC_AMR_CARD_mod[real_name_columns].sum(axis=1)
            df_ET_BC_AMR_CARD_mod_2[real_name]=sum_of_real_name_columns
            
#delate columns of thr form gen_number
for name in df_ET_BC_AMR_CARD_mod.columns[1:]:
    del df_ET_BC_AMR_CARD_mod_2[name]
print(df_ET_BC_AMR_CARD_mod_2.head())

       db  ARO:3000516  ARO:3000792  ARO:3003550  ARO:3004588  ARO:3000027  \
0  BAL_En            1            0            0            0            0   
1  BAL_Es            0            0            0            0            0   
2  BAL_Kl            0            0            0            0            0   
3  DEN_En            0            0            0            0            0   
4  DEN_Es            0            0            0            0            0   

   ARO:3000823  ARO:3000833  ARO:3000676  ARO:3005044  ...  ARO:3005059  \
0            0            0            0            0  ...            0   
1            0            0            0            0  ...            0   
2            0            0            0            0  ...            0   
3            0            0            0            0  ...            0   
4            0            0            0            0  ...            0   

   ARO:3003922  ARO:3003952  ARO:3000216  ARO:3001216  ARO:3000828  \
0         

We got it, bouth AMR-CARD and AMR-BLASTn tables have "the same format"

Now lets horizontally combined the two datasets, so that for each city we will have 325+41 =366 AMR IDS

In [10]:
merged_ETBCMet_df = pd.merge(df_ET_BC_AMR_CARD_mod_2, df_ET_BC_AMR_BLASTn, on='db')
merged_ETBCMet_df.rename(columns={'db': 'ID'}, inplace=True)
print(merged_ETBCMet_df.head())

       ID  ARO:3000516  ARO:3000792  ARO:3003550  ARO:3004588  ARO:3000027  \
0  BAL_En            1            0            0            0            0   
1  BAL_Es            0            0            0            0            0   
2  BAL_Kl            0            0            0            0            0   
3  DEN_En            0            0            0            0            0   
4  DEN_Es            0            0            0            0            0   

   ARO:3000823  ARO:3000833  ARO:3000676  ARO:3005044  ...  chuy  flii  papk  \
0            0            0            0            0  ...     0     1     0   
1            0            0            0            0  ...     0     0     0   
2            0            0            0            0  ...     0     0     0   
3            0            0            0            0  ...     0     0     0   
4            0            0            0            0  ...     0     0     0   

   papj  mrkh  flia  ybts  clbo  ibea  papc  
0   

Now we joint vertically mysterious sample (146 rows) with oru AMR jointed table (18 rows)

In [11]:
concatenated_df = pd.concat([df_CM_mysterious, merged_ETBCMet_df], axis=0, ignore_index=True).fillna(0)
print(concatenated_df.head())


   ID  ARO:3000167  ARO:3002847  mrkb  enta  fepa  ARO:3004122  cyca  csge  \
0  S1          0.0          0.0     0     0     0            0     1     0   
1  S2          0.0          0.0     0     0     0            0     1     0   
2  S3          0.0          0.0     0     0     0            0     1     0   
3  S4          0.0          0.0     0     1     0            0     1     0   
4  S5          0.0          0.0     0     0     0            0     1     0   

   iucc  ...  teml100  scintssj  wcst  sfax  kpsd  mdlb  ARO:3001084  tssj  \
0     0  ...        0         0     0     0     0     0          0.0     0   
1     0  ...        0         0     0     0     0     0          0.0     0   
2     0  ...        0         0     0     0     0     0          0.0     0   
3     0  ...        0         0     0     0     0     0          0.0     0   
4     0  ...        0         0     0     0     0     0          0.0     0   

   icmftssm  acrr  
0         0     0  
1         0     0  
2 

However. We want to extend that table with metadata of the mysterious ssample

In [12]:
#we use meta data information to add in our table
concatenated_df2 = pd.concat([mysterious_df, merged_ETBCMet_df], axis=0, ignore_index=True).fillna(0)

for i in range(len(concatenated_df2["ID"]))[-18:]:
    name = concatenated_df2["ID"][i]
    concatenated_df2["City"][i]=name.split("_")[0]
    if name.split("_")[1]=="En":
        concatenated_df2["Species"][i]= "Enterobacter hormaechei"
        #
        #eso si quieres quitar el nombre del genero de la ciudad
    elif name.split("_")[1]=="Es":
        concatenated_df2["Species"][i]= "Escherichia coli"
        #concatenated_df2["ID"][i]=name.split("_")[0]
    elif name.split("_")[1]=="Kl":
        concatenated_df2["Species"][i]= "Klebsiella pneumoniae"
        #concatenated_df2["ID"][i]=name.split("_")[0]
    concatenated_df2["ID"][i]="S"+str(i+1)
print(concatenated_df2.head())

   ID                  Species        City AST-based group  ARO:3000167  \
0  S1  Enterobacter hormaechei  mysterious   3GC-resistant          0.0   
1  S2  Enterobacter hormaechei  mysterious          CP CRE          0.0   
2  S3  Enterobacter hormaechei  mysterious          CP CRE          0.0   
3  S4  Enterobacter hormaechei  mysterious          CP CRE          0.0   
4  S5  Enterobacter hormaechei  mysterious          CP CRE          0.0   

   ARO:3002847  mrkb  enta  fepa  ARO:3004122  ...  teml100  scintssj  wcst  \
0          0.0     0     0     0            0  ...        0         0     0   
1          0.0     0     0     0            0  ...        0         0     0   
2          0.0     0     0     0            0  ...        0         0     0   
3          0.0     0     1     0            0  ...        0         0     0   
4          0.0     0     0     0            0  ...        0         0     0   

   sfax  kpsd  mdlb  ARO:3001084  tssj  icmftssm  acrr  
0     0     0    

/tmp/ipykernel_2147886/4245227815.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df2["City"][i]=name.split("_")[0]
/tmp/ipykernel_2147886/4245227815.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df2["Species"][i]= "Enterobacter hormaechei"
/tmp/ipykernel_2147886/4245227815.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df2["ID"][i]="S"+str(i+1)
/tmp/ipykernel_2147886/4245227815.py

In [13]:
#print(concatenated_df2.tail(20))